In [14]:
import cv2
import os, glob
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import random

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ConvLSTM2D, Dropout, LSTM

In [3]:
orig_img_targ = r'D:\datasets\Celeb_df_as_imgs\Original'
fake_img_targ = r'D:\datasets\Celeb_df_as_imgs\Fake'

In [4]:
orig_dir = r'D:\datasets\Celeb-DF\Celeb-real'
fake_dir = r'D:\datasets\Celeb-DF\Celeb-synthesis'
orig_dir_yt = r'D:\datasets\Celeb-DF\YouTube-real'

In [5]:
orig_files = glob.glob(os.path.join(orig_dir, '*mp4'))
fake_files = glob.glob(os.path.join(fake_dir, '*mp4'))

In [6]:
orig_files_yt = glob.glob(os.path.join(orig_dir_yt, '*mp4'))

In [7]:
print(len(orig_files))
print(len(fake_files))
print(len(orig_files_yt))

158
795
250


In [8]:
orig_files = orig_files+orig_files_yt

In [9]:
print(len(orig_files))
print(len(fake_files))

408
795


In [10]:
trial_orig = orig_files[:300]
trial_fakes = fake_files[:300]

In [15]:
def extract_faces_from_video(video_path, output_directory='output_faces'):
    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Check if the video file opened successfully
    if not video_capture.isOpened():
        print("Error: Unable to open video file.")
        return None

    # Create a directory to save the face images
    os.makedirs(output_directory, exist_ok=True)
    path_list = []
    # Loop through each frame of the video
    frame_count = 0
    while True:
        # Read the current frame
        ret, frame = video_capture.read()

        # If there are no more frames, break out of the loop
        if not ret:
            break

        # Convert the frame to RGB (MediaPipe requires RGB input)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform face detection
        results = face_detection.process(frame_rgb)

        # Draw bounding boxes around the detected faces
        if results.detections:
            for detection in results.detections:
                # Get the bounding box coordinates
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                             int(bboxC.width * iw), int(bboxC.height * ih)

                # Calculate expanded bounding box dimensions
                # head_width = int(w * 2.0)  # 50% wider
                # head_height = int(h * 2.5)  # 100% taller

                # # Adjust bounding box position to ensure it remains within frame boundaries
                # x = max(0, x - (head_width - w) // 2)
                # y = max(0, y - (head_height - h) // 2)
                # head_width = min(iw - x, head_width)
                # head_height = min(ih - y, head_height)

                # Crop the bounding box from the frame
                # face_image = frame[y:y + head_height, x:x + head_width]

                # Save the cropped face image
                face_filename = os.path.join(output_directory, f'face_{frame_count}.jpg')
                # cv2.imwrite(face_filename, face_image)
                path_list.append(face_filename)
                # Increment the frame count for naming the next face image
                frame_count += 1

    # Release the video capture object and close all windows
    video_capture.release()
    cv2.destroyAllWindows()
    print("frames = ", frame_count)
    # return output_directory
    return path_list


In [16]:
video_paths_as_imgs_orig = []

for i in range(len(trial_orig)):
    returned_list = extract_faces_from_video(video_path = trial_orig[i], output_directory = os.path.join(r'D:\datasets\Tharun na proj\Originals', str(i)))
    video_paths_as_imgs_orig.append(returned_list)
    print(i, "  done")

frames =  404
0   done
frames =  303
1   done
frames =  350
2   done
frames =  529
3   done
frames =  326
4   done
frames =  459
5   done
frames =  534
6   done
frames =  479
7   done
frames =  464
8   done
frames =  462
9   done
frames =  239
10   done
frames =  148
11   done
frames =  322
12   done
frames =  343
13   done
frames =  517
14   done
frames =  318
15   done
frames =  248
16   done
frames =  490
17   done
frames =  380
18   done
frames =  536
19   done
frames =  363
20   done
frames =  441
21   done
frames =  357
22   done
frames =  313
23   done
frames =  323
24   done
frames =  327
25   done
frames =  316
26   done
frames =  320
27   done
frames =  371
28   done
frames =  397
29   done
frames =  330
30   done
frames =  376
31   done
frames =  409
32   done
frames =  364
33   done
frames =  280
34   done
frames =  339
35   done
frames =  314
36   done
frames =  325
37   done
frames =  327
38   done
frames =  349
39   done
frames =  400
40   done
frames =  340
41   done
fr

In [31]:
def frame_skipper(paths_list):
    new_list = []
    for i in range(len(paths_list)):
        if i%2 ==0:
            new_list.append(paths_list[i])
        if i>=199:
            break

    return new_list

In [32]:
print(len(frame_skipper(video_paths_as_imgs_orig[1])))

100


In [33]:
videoframe_paths_orig_final = []
for i in range(len(video_paths_as_imgs_orig)):
    returned_list = frame_skipper(video_paths_as_imgs_orig[i])
    videoframe_paths_orig_final.append(returned_list)

['D:\\datasets\\Tharun na proj\\Originals\\34\\face_0.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_2.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_4.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_6.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_8.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_10.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_12.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_14.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_16.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_18.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_20.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_22.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_24.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_26.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_28.jpg',
 'D:\\datasets\\Tharun na proj\\Originals\\34\\face_30.jpg',
 'D:\\datasets\\Tharun na pro

In [34]:
a = videoframe_paths_orig_final[0][0]
img = cv2.imread(a)
print(img.shape)

(350, 280, 3)


In [23]:
print(len(a))

49


In [41]:
for i in range(len(videoframe_paths_orig_final)):
    for j in range(len(videoframe_paths_orig_final[i])):
        path = videoframe_paths_orig_final[i][j]
        img = cv2.imread(path)
        img = cv2.resize(img, (150, 150))
        cv2.imwrite(os.path.join(orig_img_targ, str(i)+'_'+str(j)+'.jpg'), img)
    print(i, '  done')

0   done
1   done
2   done
3   done
4   done
5   done
6   done
7   done
8   done
9   done
10   done
11   done
12   done
13   done
14   done
15   done
16   done
17   done
18   done
19   done
20   done
21   done
22   done
23   done
24   done
25   done
26   done
27   done
28   done
29   done
30   done
31   done
32   done
33   done
34   done
35   done
36   done
37   done
38   done
39   done
40   done
41   done
42   done
43   done
44   done
45   done
46   done
47   done
48   done
49   done
50   done
51   done
52   done
53   done
54   done
55   done
56   done
57   done
58   done
59   done
60   done
61   done
62   done
63   done
64   done
65   done
66   done
67   done
68   done
69   done
70   done
71   done
72   done
73   done
74   done
75   done
76   done
77   done
78   done
79   done
80   done
81   done
82   done
83   done
84   done
85   done
86   done
87   done
88   done
89   done
90   done
91   done
92   done
93   done
94   done
95   done
96   done
97   done
98   done
99   done
100   done

In [37]:
video_paths_as_imgs_fakes = []

for i in range(len(trial_fakes)):
    returned_list = extract_faces_from_video(video_path = trial_fakes[i], output_directory = os.path.join(r'D:\datasets\Tharun na proj\Fakes', str(i)))
    video_paths_as_imgs_fakes.append(returned_list)
    print(i, "  done")

frames =  422
0   done
frames =  303
1   done
frames =  350
2   done
frames =  529
3   done
frames =  326
4   done
frames =  459
5   done
frames =  534
6   done
frames =  479
7   done
frames =  464
8   done
frames =  465
9   done
frames =  418
10   done
frames =  303
11   done
frames =  350
12   done
frames =  529
13   done
frames =  459
14   done
frames =  534
15   done
frames =  479
16   done
frames =  468
17   done
frames =  393
18   done
frames =  303
19   done
frames =  350
20   done
frames =  529
21   done
frames =  459
22   done
frames =  534
23   done
frames =  479
24   done
frames =  464
25   done
frames =  401
26   done
frames =  303
27   done
frames =  350
28   done
frames =  529
29   done
frames =  326
30   done
frames =  459
31   done
frames =  534
32   done
frames =  479
33   done
frames =  464
34   done
frames =  464
35   done
frames =  414
36   done
frames =  303
37   done
frames =  350
38   done
frames =  529
39   done
frames =  326
40   done
frames =  459
41   done
fr

In [38]:
videoframe_paths_fake_final = []
for i in range(len(video_paths_as_imgs_fakes)):
    returned_list = frame_skipper(video_paths_as_imgs_fakes[i])
    videoframe_paths_fake_final.append(returned_list)

In [39]:
videoframe_paths_fake_final[0]

['D:\\datasets\\Tharun na proj\\Fakes\\0\\face_0.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_2.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_4.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_6.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_8.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_10.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_12.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_14.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_16.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_18.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_20.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_22.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_24.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_26.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_28.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_30.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_32.jpg',
 'D:\\datasets\\Tharun na proj\\Fakes\\0\\face_34.jpg

In [43]:
for i in range(len(videoframe_paths_fake_final)):
    for j in range(len(videoframe_paths_fake_final[i])):
        path = videoframe_paths_fake_final[i][j]
        img = cv2.imread(path)
        img = cv2.resize(img, (150, 150))
        cv2.imwrite(os.path.join(fake_img_targ, str(i)+'_'+str(j)+'.jpg'), img)
    print(i, '  done')

0   done
1   done
2   done
3   done
4   done
5   done
6   done
7   done
8   done
9   done
10   done
11   done
12   done
13   done
14   done
15   done
16   done
17   done
18   done
19   done
20   done
21   done
22   done
23   done
24   done
25   done
26   done
27   done
28   done
29   done
30   done
31   done
32   done
33   done
34   done
35   done
36   done
37   done
38   done
39   done
40   done
41   done
42   done
43   done
44   done
45   done
46   done
47   done
48   done
49   done
50   done
51   done
52   done
53   done
54   done
55   done
56   done
57   done
58   done
59   done
60   done
61   done
62   done
63   done
64   done
65   done
66   done
67   done
68   done
69   done
70   done
71   done
72   done
73   done
74   done
75   done
76   done
77   done
78   done
79   done
80   done
81   done
82   done
83   done
84   done
85   done
86   done
87   done
88   done
89   done
90   done
91   done
92   done
93   done
94   done
95   done
96   done
97   done
98   done
99   done
100   done

In [44]:
trial_orig_test = orig_files[300:400]
trial_fakes_test = fake_files[300:400]

In [45]:
video_paths_as_imgs_orig = []

for i in range(len(trial_orig_test)):
    returned_list = extract_faces_from_video(video_path = trial_orig_test[i], output_directory = os.path.join(r'D:\datasets\Tharun na proj\Originals', str(i)))
    video_paths_as_imgs_orig.append(returned_list)
    print(i, "  done")

frames =  450
0   done
frames =  634
1   done
frames =  451
2   done
frames =  469
3   done
frames =  722
4   done
frames =  514
5   done
frames =  459
6   done
frames =  506
7   done
frames =  461
8   done
frames =  504
9   done
frames =  698
10   done
frames =  488
11   done
frames =  88
12   done
frames =  573
13   done
frames =  445
14   done
frames =  443
15   done
frames =  455
16   done
frames =  448
17   done
frames =  490
18   done
frames =  920
19   done
frames =  448
20   done
frames =  475
21   done
frames =  87
22   done
frames =  489
23   done
frames =  469
24   done
frames =  477
25   done
frames =  476
26   done
frames =  432
27   done
frames =  348
28   done
frames =  436
29   done
frames =  353
30   done
frames =  499
31   done
frames =  465
32   done
frames =  469
33   done
frames =  605
34   done
frames =  521
35   done
frames =  468
36   done
frames =  451
37   done
frames =  349
38   done
frames =  412
39   done
frames =  478
40   done
frames =  281
41   done
fram

In [46]:
print(len(frame_skipper(video_paths_as_imgs_orig[1])))

100


In [53]:
videoframe_paths_orig_final = []
for i in range(len(video_paths_as_imgs_orig)):
    returned_list = frame_skipper(video_paths_as_imgs_orig[i])
    videoframe_paths_orig_final.append(returned_list)

In [58]:
for i in range(len(videoframe_paths_orig_final)):
    for j in range(len(videoframe_paths_orig_final[i])):
        path = videoframe_paths_orig_final[i][j]
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            cv2.imwrite(os.path.join(test_orig_target, str(i), str(i)+'_'+str(j)+'.jpg'), img)
    print(i, '  done')

0   done
1   done
2   done
3   done
4   done
5   done
6   done
7   done
8   done
9   done
10   done
11   done
12   done
13   done
14   done
15   done
16   done
17   done
18   done
19   done
20   done
21   done
22   done
23   done
24   done
25   done
26   done
27   done
28   done
29   done
30   done
31   done
32   done
33   done
34   done
35   done
36   done
37   done
38   done
39   done
40   done
41   done
42   done
43   done
44   done
45   done
46   done
47   done
48   done
49   done
50   done
51   done
52   done
53   done
54   done
55   done
56   done
57   done
58   done
59   done
60   done
61   done
62   done
63   done
64   done
65   done
66   done
67   done
68   done
69   done
70   done
71   done
72   done
73   done
74   done
75   done
76   done
77   done
78   done
79   done
80   done
81   done
82   done
83   done
84   done
85   done
86   done
87   done
88   done
89   done
90   done
91   done
92   done
93   done
94   done
95   done
96   done
97   done
98   done
99   done


In [48]:
test_orig_target = r'D:\datasets\Celeb_df_as_imgs\Test\Original'

In [57]:
for i in range(len(video_paths_as_imgs_orig)):
    directory = os.path.join(test_orig_target, str(i))
    if not os.path.exists(directory):
    # Create the directory
        os.mkdir(directory)
        print("Directory", directory, "created successfully")
    else:
        print("Directory", directory, "already exists")


Directory D:\datasets\Celeb_df_as_imgs\Test\Original\0 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\1 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\2 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\3 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\4 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\5 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\6 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\7 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\8 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\9 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\10 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\11 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Original\12 created successfully
Directory

In [66]:
test_fake_target = r'D:\datasets\Celeb_df_as_imgs\Test\Fake'

In [64]:
video_paths_as_imgs_fakes = []

for i in range(len(trial_fakes_test)):
    returned_list = extract_faces_from_video(video_path = trial_fakes[i], output_directory = os.path.join(r'D:\datasets\Tharun na proj\Fakes', str(i)))
    video_paths_as_imgs_fakes.append(returned_list)
    print(i, "  done")

frames =  422
0   done
frames =  303
1   done
frames =  350
2   done
frames =  529
3   done
frames =  326
4   done
frames =  459
5   done
frames =  534
6   done
frames =  479
7   done
frames =  464
8   done
frames =  465
9   done
frames =  418
10   done
frames =  303
11   done
frames =  350
12   done
frames =  529
13   done
frames =  459
14   done
frames =  534
15   done
frames =  479
16   done
frames =  468
17   done
frames =  393
18   done
frames =  303
19   done
frames =  350
20   done
frames =  529
21   done
frames =  459
22   done
frames =  534
23   done
frames =  479
24   done
frames =  464
25   done
frames =  401
26   done
frames =  303
27   done
frames =  350
28   done
frames =  529
29   done
frames =  326
30   done
frames =  459
31   done
frames =  534
32   done
frames =  479
33   done
frames =  464
34   done
frames =  464
35   done
frames =  414
36   done
frames =  303
37   done
frames =  350
38   done
frames =  529
39   done
frames =  326
40   done
frames =  459
41   done
fr

In [68]:
for i in range(len(video_paths_as_imgs_fakes)):
    directory = os.path.join(test_fake_target, str(i))
    if not os.path.exists(directory):
    # Create the directory
        os.mkdir(directory)
        print("Directory", directory, "created successfully")
    else:
        print("Directory", directory, "already exists")


Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\0 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\1 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\2 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\3 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\4 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\5 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\6 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\7 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\8 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\9 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\10 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\11 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\12 created successfully
Directory D:\datasets\Celeb_df_as_imgs\Test\Fake\13 created s

In [69]:
videoframe_paths_fake_final = []
for i in range(len(video_paths_as_imgs_fakes)):
    returned_list = frame_skipper(video_paths_as_imgs_fakes[i])
    videoframe_paths_fake_final.append(returned_list)

In [70]:
for i in range(len(videoframe_paths_fake_final)):
    for j in range(len(videoframe_paths_fake_final[i])):
        path = videoframe_paths_fake_final[i][j]
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            cv2.imwrite(os.path.join(test_fake_target, str(i), str(i)+'_'+str(j)+'.jpg'), img)
    print(i, '  done')

0   done
1   done
2   done
3   done
4   done
5   done
6   done
7   done
8   done
9   done
10   done
11   done
12   done
13   done
14   done
15   done
16   done
17   done
18   done
19   done
20   done
21   done
22   done
23   done
24   done
25   done
26   done
27   done
28   done
29   done
30   done
31   done
32   done
33   done
34   done
35   done
36   done
37   done
38   done
39   done
40   done
41   done
42   done
43   done
44   done
45   done
46   done
47   done
48   done
49   done
50   done
51   done
52   done
53   done
54   done
55   done
56   done
57   done
58   done
59   done
60   done
61   done
62   done
63   done
64   done
65   done
66   done
67   done
68   done
69   done
70   done
71   done
72   done
73   done
74   done
75   done
76   done
77   done
78   done
79   done
80   done
81   done
82   done
83   done
84   done
85   done
86   done
87   done
88   done
89   done
90   done
91   done
92   done
93   done
94   done
95   done
96   done
97   done
98   done
99   done
